Task requirements as define on the article, and they are: Horizon, Completeness, Real time, Windows Size, Accuracy measure, Interpretability, Statistical priority
Please, set each of those parameters for the analysis.

In [1]:
from ipywidgets import interact, widgets
from IPython.display import display, HTML
import os
from TSAF import problemDefinition as problem
import pandas as pd
from TSAF.base import Series
import json

%matplotlib widget


def perform_analysis(object):
    site = 'NRSMAI'
    variable = 'TEMP'
    depth= '20'
    print('processing request')
    dir = os.getcwd()
    data_path = os.path.join(dir, 'data', )
    files_list = os.listdir(data_path)
    files_list2 = filter(lambda x: site in x and variable in x and depth+".json" in x, files_list)
    for file_name in files_list2:
        if file_name.find(site) > -1 and file_name.find(variable) > -1 and file_name.find(depth+".json") > -1:
            file_name_with_path = os.path.join(data_path,file_name)
            timeSeries = problem.load_json_time_series(file_name_with_path)
            summary = problem.summary_series(timeSeries)
            detrend = timeSeries.detrend
            print(detrend)
            summary['stationary'] = problem.identify_time_series_characteristics(timeSeries)
            summary['linear_seasonal_stationary'] = problem.identify_time_series_characteristics(problem.remove_seasonality(timeSeries,'linear'))
            summary['polynomial_seasonal_stationary'] = problem.identify_time_series_characteristics(problem.remove_seasonality(timeSeries,'poly'))
            summary['difference_stationary'] = problem.identify_time_series_characteristics(problem.remove_seasonality(timeSeries,'diff'))
            summary['horizon'] = horizon.value
            summary['longterm_simulation'] = type_of_long_term_simulation.value
            summary['completeness'] = completeness.value
            summary['real_time']=real_time.value
            summary['windows'] = windows.value
            summary['accuracy'] = accuracy.value
            summary['interpretability'] = interpretability.value
            summary['priority'] = priority.value
            data_path = os.path.join(dir, 'results', )
            print(summary)
            file_name_with_path=os.path.join(data_path,'system_requirements_'+site+'_'+variable+'_'+depth+'.json')
            with open(file_name_with_path, "w") as outfile:
                json.dump(summary, outfile)



horizon = widgets.IntText(
    value=1,
    description='Horizon (t+):',
    disabled=False
)
type_of_long_term_simulation = widgets.Dropdown(
   options=['sequential','point'],
   description='Type of long term forecasting',
   value = 'sequential',
   disabled=False
)
completeness = widgets.Dropdown(
    options=['measurement','back fill'],
    description='missing value treatment:',
    value = 'measurement',
    disabled=False
)
real_time = widgets.Dropdown(
    options=['batch','real time'],
    value='batch',
    description='processing type:',
    disabled=False
)
windows = widgets.IntText(
    value=100,
    description='Windows Size:',
    disabled=False
)
accuracy = widgets.SelectMultiple(
    options=['RMSE', 'MAE', 'MDAE', 'MAPE', 'CC', 'PE','MDAPE', 'MASE', 'RELMAE', 'R^2'],
    #rows=10,
    description='compulsory quality measures:',
    disabled=False
)
interpretability = widgets.Dropdown(
    options=['structural','white box','black box'],
    value='black box',
    description='minimum interpretability provided',
    disabled=False
)
priority  = widgets.Dropdown(
     options=['statistical validity','model variety'],
     value='statistical validity',
     description='priority',
     disabled=False
 )

button = widgets.Button(
    description='generate requirements',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='generate requirements ',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

a=widgets.HBox([horizon, type_of_long_term_simulation, completeness])
b=widgets.HBox([real_time, windows, accuracy])
c=widgets.HBox([interpretability,priority, button])
vertical_input = widgets.VBox([a, b,c])
display(vertical_input)
button.on_click(perform_analysis)

The design step process the output file system_requirements and compare with models, pre-processing and measures to define the best combination for the design. Aditionally, the user can inspect and adjust the design file.

benchmarks stationary:
Naive
ARMA
ETS (no missing)

benchmarks non stationary:
Polynomial naive
ARIMA
STL (1 harmonic) (no missing)
ETS (no missing)


Base competitors:
GAM
KNN
BSTS
SVR*
ANN*
LSTM*
GRU*
Convolutional*
[not available] Deep state space model*
[not available] Deep factor model*
(*black box)




In [2]:
#import data properties files
import pandas as pd
print(os.path.dirname(problem.__file__))
package_dir = os.path.dirname(problem.__file__)
data_path = os.path.join(package_dir, 'data', )
methods_file_name = 'methods.csv'
measures_file = 'error_measures.csv'
requirements = '/Users/viniciussantino/OneDrive - James Cook University/PhD/Code/TSAF/example/results/system_requirements_NRSMAI_TEMP_20.json'
file_name_with_path=os.path.join(data_path,methods_file_name)
df = pd.read_csv(file_name_with_path, delimiter=";",index_col=0)

#print(df.columns)
df[['benchmark','missing','non stationary','complex seasonality','linear']] = df[['benchmark','missing','non stationary','complex seasonality','linear']].applymap(lambda x:  x == 'Yes')
display(df)

with open(requirements) as json_file:
    requirements_content = json.load(json_file)

#filter methods by interpretability
(df2,df3,df4) = problem.filter_parameters(methods=df, measures=df,filters=requirements_content)

display(df2)


#df.filter(lambda x:)
#filter pre-processing methods
#filter methods
#prioratise measures

/Users/viniciussantino/OneDrive - James Cook University/PhD/Code/TSAF/TSAF
{'observation': 411327, 'beginning': 1219924852.08, 'end': 1590118254.548, 'period': 900.0, 'null_observations': 37154, 'continuous_observations': 20337, 'has_zeros': 'False', 'stationary': {'stationary': 'No'}, 'linear_seasonal_stationary': {'stationary': 'No'}, 'polynomial_seasonal_stationary': {'stationary': 'Sample', 'diff': True, 'trend': False}, 'difference_stationary': {'stationary': 'Population', 'diff': False, 'trend': False}, 'horizon': 1, 'longterm_simulation': 'sequential', 'completeness': 'measurement', 'real_time': 'batch', 'windows': 100, 'accuracy': [], 'interpretability': 'black box', 'priority': 'statistical validity'}
Index(['benchmark', 'intepretability', 'missing', 'non stationary',
       'complex seasonality', 'complexity', 'linear'],
      dtype='object')


,benchmark,intepretability,missing,non stationary,complex seasonality,complexity,linear
method,,,,,,,
naive,True,"white box,black box,structural",True,False,False,-1,True
ARMA,True,"white box,black box,structural",True,False,False,1,True
ETS,True,"white box,black box,structural",False,True,True,2,True
Polynomial naive,True,"white box,black box,structural",True,True,True,0,True
ARIMA,True,"white box,black box,structural",True,True,False,2,True
STL,True,"white box,black box,structural",True,True,False,2,True
Profet,False,"white box,black box,structural",True,True,True,3,True
KNN,False,"white box,black box",True,True,True,3,False
BSTS,False,"white box,black box,structural",True,True,True,3,True


,benchmark,intepretability,missing,non stationary,complex seasonality,complexity,linear,valid_interpretability,valid_missing
method,,,,,,,,,
naive,True,"white box,black box,structural",True,False,False,-1,True,True,True
ARMA,True,"white box,black box,structural",True,False,False,1,True,True,True
Polynomial naive,True,"white box,black box,structural",True,True,True,0,True,True,True
ARIMA,True,"white box,black box,structural",True,True,False,2,True,True,True
STL,True,"white box,black box,structural",True,True,False,2,True,True,True
Profet,False,"white box,black box,structural",True,True,True,3,True,True,True
KNN,False,"white box,black box",True,True,True,3,False,True,True
BSTS,False,"white box,black box,structural",True,True,True,3,True,True,True
SVR,False,black box,True,True,True,4,False,True,True
